In [1]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl (8.9 MB)
Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- ------------------

In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib  # to save the model later


In [3]:
# df_model = pd.read_csv("my_data.csv")
# df_model.head()

In [4]:
# df_model.shape

In [5]:
# # Assuming df_model has "symptoms_text" and "diseases" columns
# X = df_model["symptoms_text"]
# y = df_model["diseases"]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )


In [6]:
# vectorizer = TfidfVectorizer(
#     max_features=3000,      # limits features for speed
#     ngram_range=(1, 2),     # captures phrases like "chest pain"
#     stop_words="english"    # remove common words
# )

# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)


In [7]:
# model = LogisticRegression(max_iter=1000)
# model.fit(X_train_tfidf, y_train)


In [8]:
# y_pred = model.predict(X_test_tfidf)
# print(classification_report(y_test, y_pred))


In [9]:
# from sklearn.metrics import accuracy_score

# # Train accuracy
# y_train_pred = model.predict(X_train_tfidf)
# train_acc = accuracy_score(y_train, y_train_pred)
# print(f"Training Accuracy: {train_acc:.4f}")

# # Test accuracy
# y_test_pred = model.predict(X_test_tfidf)
# test_acc = accuracy_score(y_test, y_test_pred)
# print(f"Test Accuracy: {test_acc:.4f}")


In [10]:
# def predict_disease(symptoms_text):
#     text_tfidf = vectorizer.transform([symptoms_text])
#     prediction = model.predict(text_tfidf)[0]
#     return prediction

# # Example:
# user_input = " anxiety and nervousness, shortness of breath,depression ,insomnia"
# print("Predicted disease:", predict_disease(user_input))


In [11]:
# import random


In [12]:

# X = df_model['symptoms_text']
# y = df_model['diseases']

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # -------------------------------
# # 4️⃣ TF-IDF vectorizer (fit only on train)
# # -------------------------------
# vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2), stop_words='english')
# X_train_tfidf = vectorizer.fit_transform(X_train)

# # -------------------------------
# # 5️⃣ Define chatbot function
# # -------------------------------
# # Rule-based greetings and fallbacks
# greetings = ["hello", "hi", "hey", "good morning", "good evening"]
# greeting_responses = [
#     "Hello! How can I help you with your symptoms today?", 
#     "Hi there! Tell me your symptoms so I can assist.",
#     "Hey! What symptoms are you experiencing?"
# ]

# fallback_responses = [
#     "I'm here to help with medical symptoms and possible diseases.",
#     "Please tell me your symptoms so I can assist you."
# ]

In [13]:
# # Main chatbot function
# def medical_chatbot(user_input):
#     user_input_lower = user_input.lower()
    
#     # 1. Greetings
#     if any(word in user_input_lower for word in greetings):
#         return random.choice(greeting_responses)
    
#     # 2. Non-medical queries 
#     symptom_keywords = ["pain", "fever", "cough", "dizzy", "headache", "shortness", "nausea"]
#     if not any(word in user_input_lower for word in symptom_keywords):
#         return random.choice(fallback_responses)
    
#     # 3. TF-IDF retrieval
#     input_vec = vectorizer.transform([user_input])
#     similarities = cosine_similarity(input_vec, X_train_tfidf).flatten()
#     idx = similarities.argmax()
    
#     return f"Based on your symptoms, you might have: {y_train.iloc[idx]}"

# #Evaluate chatbot performan
# y_pred = [medical_chatbot(text) for text in X_test]

# # Remove "Based on your symptoms, you might have: " prefix to match labels
# y_pred_clean = [pred.replace("Based on your symptoms, you might have: ", "") for pred in y_pred]


In [14]:

# y_pred_clean = [pred.replace("Based on your symptoms, you might have: ", "") for pred in y_pred]

# accuracy = accuracy_score(y_test, y_pred_clean)
# print(f"Test Accuracy: {accuracy:.4f}\n")
# print("Classification Report:")
# print(classification_report(y_test, y_pred_clean))
      
# # Optional: interactive chat loop
# print("\nMedical Chatbot: type 'exit' to quit")
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == "exit":
#         print("Chatbot: Goodbye!")
#         break
#     response = medical_chatbot(user_input)
#     print("Chatbot:", response)

## RANDOM FOREST 

In [16]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report
# import random

# # ------------------------------------------
# # 1. Load data
# # ------------------------------------------
# # df_model has: 'symptoms_text' and 'diseases'
# df = df_model.copy()

# # ------------------------------------------
# # 2. Train-test split
# # ------------------------------------------
# X_train, X_test, y_train, y_test = train_test_split(
#     df["symptoms_text"],
#     df["diseases"],
#     test_size=0.2,
#     random_state=42,
#     stratify=df["diseases"]
# )


In [17]:
# # ------------------------------------------
# # 3. TF-IDF vectorization
# # ------------------------------------------
# vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2), stop_words="english")
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf  = vectorizer.transform(X_test)

# # 4. Train Random Forest

# rf_model = RandomForestClassifier(
#     n_estimators=300,
#     max_depth=20,
#     random_state=42,
#     class_weight="balanced",
#     n_jobs=-1
# )
# rf_model.fit(X_train_tfidf, y_train)

In [18]:
# # 5. Evaluate performance
# # ------------------------------------------
# y_train_pred = rf_model.predict(X_train_tfidf)
# y_test_pred  = rf_model.predict(X_test_tfidf)

# print(f"✅ Training Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
# print(f"✅ Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}\n")
# print("Classification Report (Test):")
# print(classification_report(y_test, y_test_pred))


In [19]:
# from sklearn.model_selection import train_test_split, cross_val_score
# X = df['symptoms_text']
# y = df['diseases']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # -----------------------------
# # 3️⃣ TF-IDF Vectorization (optimized)
# # -----------------------------
# vectorizer = TfidfVectorizer(
#     max_features=5000,
#     ngram_range=(1,3),
#     stop_words='english',
#     sublinear_tf=True
# )
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # -----------------------------
# # 4️⃣ Random Forest Model (tuned)
# # -----------------------------
# rf_model = RandomForestClassifier(
#     n_estimators=300,
#     max_depth=15,
#     min_samples_split=5,
#     min_samples_leaf=3,
#     class_weight='balanced',
#     random_state=42,
#     n_jobs=-1
# )

# # -----------------------------
# # 5️⃣ Cross-validation (5-fold)
# # -----------------------------
# cv_scores = cross_val_score(rf_model, X_train_tfidf, y_train, cv=5, scoring='accuracy')
# print(f"Average CV Accuracy: {cv_scores.mean():.4f}")


In [20]:
# # -----------------------------
# # 6️⃣ Train final model
# # -----------------------------
# rf_model.fit(X_train_tfidf, y_train)

# # -----------------------------
# # 7️⃣ Evaluate performance
# # -----------------------------
# y_pred_train = rf_model.predict(X_train_tfidf)
# y_pred_test = rf_model.predict(X_test_tfidf)

# train_acc = accuracy_score(y_train, y_pred_train)
# test_acc = accuracy_score(y_test, y_pred_test)

# print(f"\n✅ Training Accuracy: {train_acc:.4f}")
# print(f"✅ Test Accuracy: {test_acc:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred_test))

In [21]:
# !pip install tensorflow


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import joblib


In [23]:

#Load data

df = pd.read_csv("my_data.csv")  # columns: symptoms_text, diseases

X = df["symptoms_text"].astype(str)
y = df["diseases"].astype(str)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# -------------------------------
# 2️⃣ Train-test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# -------------------------------
# 3️⃣ Tokenization & Padding
MAX_WORDS = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post", truncating="post")


In [24]:
# -------------------------------
# 4️⃣ Neural Network (Lightweight)
# -------------------------------
model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN),
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.2, return_sequences=False)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# -------------------------------
# 5️⃣ Train model
# -------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
2287/2287 [==============================] - 390s 168ms/step - loss: 1.5715 - accuracy: 0.5300 - val_loss: 0.5239 - val_accuracy: 0.8039
Epoch 2/10
2287/2287 [==============================] - 366s 160ms/step - loss: 0.5881 - accuracy: 0.7963 - val_loss: 0.3841 - val_accuracy: 0.8523
Epoch 3/10
2287/2287 [==============================] - 369s 161ms/step - loss: 0.4701 - accuracy: 0.8314 - val_loss: 0.3320 - val_accuracy: 0.8648
Epoch 4/10
2287/2287 [==============================] - 381s 166ms/step - loss: 0.4120 - accuracy: 0.8458 - val_loss: 0.3087 - val_accuracy: 0.8701
Epoch 5/10
2287/2287 [==============================] - 389s 170ms/step - loss: 0.3816 - accuracy: 0.8529 - val_loss: 0.2943 - val_accuracy: 0.8739
Epoch 6/10
2287/2287 [==============================] - 411s 180ms/step - loss: 0.3576 - accuracy: 0.8594 - val_loss: 0.2911 - val_accuracy: 0.8717
Epoch 7/10
2287/2287 [==============================] - 409s 179ms/step - loss: 0.3431 - accuracy: 0.8620 - val_

In [25]:
# Evaluate performance
train_loss, train_acc = model.evaluate(X_train_pad, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=0)

print(f"✅ Training Accuracy: {train_acc:.4f}")
print(f"✅ Test Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
y_pred = np.argmax(model.predict(X_test_pad), axis=1)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


✅ Training Accuracy: 0.8854
✅ Test Accuracy: 0.8805

Classification Report:
715/715 [==============================] - 14s 19ms/step
                                             precision    recall  f1-score   support

                          actinic_keratosis       0.96      0.68      0.79       182
                        acute_bronchiolitis       0.94      0.92      0.93       241
                           acute_bronchitis       0.79      0.76      0.77       243
                         acute_bronchospasm       0.72      0.74      0.73       181
                        acute_kidney_injury       0.96      0.98      0.97       182
                         acute_pancreatitis       0.84      0.97      0.90       241
                            acute_sinusitis       0.79      0.95      0.86       181
                      acute_stress_reaction       0.93      0.92      0.93       182
                                    allergy       0.97      0.98      0.97       181
                

In [26]:
# Save model and tokenizer
model.save("medical_nn_model.h5")
joblib.dump(tokenizer, "tokenizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
print("\n Model, tokenizer, and label encoder saved successfully!")


 Model, tokenizer, and label encoder saved successfully!


C:\Users\USER\anaconda3\envs\felix\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
from tensorflow.keras.saving import saving_api
from tensorflow.keras.models import load_model

model = saving_api.load_model('medical_nn_model.h5', compile=True)


ImportError: cannot import name 'saving_api' from 'tensorflow.keras.saving' (C:\Users\USER\anaconda3\envs\felix\lib\site-packages\keras\api\_v2\keras\saving\__init__.py)

In [27]:
# def medical_chatbot(user_input):
#     import random
#     greetings = ["hello", "hi", "hey", "good morning", "good evening"]
#     responses = ["Hello! How can I assist you today?", "Hi there! What symptoms are you experiencing?"]
    
#     if any(g in user_input.lower() for g in greetings):
#         return random.choice(responses)
    
#     seq = tokenizer.texts_to_sequences([user_input])
#     pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post')
#     pred = model.predict(pad)
#     disease = label_encoder.inverse_transform([np.argmax(pred)])[0]
#     return f"Based on your symptoms, you might have: {disease}"

# # -------------------------------
# # 💬 Try the chatbot
# print("\nChatbot ready! Type 'exit' to stop.")
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == "exit":
#         print("Chatbot: Goodbye!")
#         break
#     print("Chatbot:", medical_chatbot(user_input))


In [28]:
# # 6. Add chatbot functionality
# # ------------------------------------------
# greetings = ["hi", "hello", "hey", "good morning", "good evening"]
# greeting_responses = ["Hello! How can I help you today?", "Hi there!", "Hey! Tell me your symptoms."]

# def medical_chatbot(user_input):
#     user_input_lower = user_input.lower()

#     # Greeting handler
#     if any(greet in user_input_lower for greet in greetings):
#         return random.choice(greeting_responses)

#     # Predict disease from input
#     input_vec = vectorizer.transform([user_input])
#     prediction = rf_model.predict(input_vec)[0]
#     return f"Based on your symptoms, you might have: {prediction}"

# # ------------------------------------------
# # 7. Interactive chat
# # ------------------------------------------
# print("\n Medical Chatbot: Type 'exit' to quit.")
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == "exit":
#         print("Chatbot: Goodbye! Take care ❤️")
#         break
#     print("Chatbot:", medical_chatbot(user_input))

In [29]:
# import pickle
# import pandas as pd

# # Save TF-IDF vectorizer
# with open("tfidf_vectorizer.pkl", "wb") as f:
#     pickle.dump(vectorizer, f)

# # Save training data
# train_data = pd.DataFrame({
#     "symptoms_text": X_train,   # or df_model['symptoms_text']
#     "diseases": y_train         # or df_model['diseases']
# })
# train_data.to_csv("train_data.csv", index=False)


In [30]:
# import pickle
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# import random

# # Load vectorizer and training data
# with open("tfidf_vectorizer.pkl", "rb") as f:
#     vectorizer = pickle.load(f)

# train_data = pd.read_csv("train_data.csv")
# X_train = train_data['symptoms_text']
# y_train = train_data['diseases']

# greetings = ["hello", "hi", "hey", "good morning", "good evening"]
# greeting_responses = [
#     "Hello! How can I help you with your symptoms today?", 
#     "Hi there! Tell me your symptoms so I can assist.",
#     "Hey! What symptoms are you experiencing?"
# ]
# fallback_responses = [
#     "I'm here to help with medical symptoms and possible diseases.",
#     "Please tell me your symptoms so I can assist you."
# ]

# def medical_chatbot(user_input):
#     user_input_lower = user_input.lower()
#     if any(word in user_input_lower for word in greetings):
#         return random.choice(greeting_responses)
    
#     symptom_keywords = ["pain", "fever", "cough", "dizzy", "headache", "shortness", "nausea"]
#     if not any(word in user_input_lower for word in symptom_keywords):
#         return random.choice(fallback_responses)
    
#     input_vec = vectorizer.transform([user_input])
    # similarities = cosine_similarity(input_vec, vectorizer.transform(X_train)).flatten()
    # idx = similarities.argmax()
    
    # return f"Based on your symptoms, you might have: {y_train.iloc[idx]}"


In [31]:
# from medical_chatbot import medical_chatbot

# while True:
#     user_input = input("You: ")
#     if user_input.lower() == "exit":
#         print("Chatbot: Goodbye!")
#         break
#     print("Chatbot:", medical_chatbot(user_input))


In [32]:
# from fastapi import FastAPI, Request
# from fastapi.middleware.cors import CORSMiddleware
# import joblib
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# import re

# #  Initialize FastAPI 
# app = FastAPI(title="Doc-AI Backend")

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["*"],
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

# # Load model, tokenizer, and encoder 
# print(" Loading model and preprocessors...")
# model = tf.keras.models.load_model("medical_nn_model.h5")
# tokenizer = joblib.load("tokenizer.pkl")
# label_encoder = joblib.load("label_encoder.pkl")
# print("Model, tokenizer, and label encoder loaded successfully!")

# # Greeting and fallback patterns 
# GREETINGS = [
#     "hi", "hello", "hey", "good morning", "good afternoon", "good evening", "what’s up", "heyy", "yo", "yoh", "how are you","Wassup","Wagwan"
# ]
# THANKS = [
#     "thank you", "thanks", "appreciate", "grateful","Okay","Ok"
# ]
# BYE = [
#     "bye", "goodbye", "see you", "talk later","later"
# ]

# # clean and preprocess text 
# def clean_text(text):
#     text = text.lower().strip()
#     text = re.sub(r"[^a-zA-Z\s]", "", text)
#     return text

# def preprocess_text(text: str):
#     seq = tokenizer.texts_to_sequences([text])
#     if not seq or len(seq[0]) == 0:
#         return None
#     return pad_sequences(seq, maxlen=100)  

# # Main prediction route 
# @app.post("/predict")
# async def predict(request: Request):
#     data = await request.json()
#     user_input = clean_text(data.get("text", "").strip())

#     if not user_input:
#         return {
#             "bot_message": "Please describe how you're feeling so I can help.",
#             "status": "error"
    #     }

    # # Handle greetings 
    # if any(greet in user_input for greet in GREETINGS):
    #     return {
    #         "bot_message": (
    #             "👋 Hey there! I’m **Doc-AI**, your health companion.\n"
    #             "Felling a bit off today?Tell me about your symptoms and I'll try to help."
    #         ),
    #         "status": "greeting"
    #     }

    # #  Handle gratitude 
    # if any(thank in user_input for thank in THANKS):
    #     return {
    #         "bot_message": (
    #             "😊 You’re most welcome! I’m here to help. "
    #             "If you have more symptoms, just tell me."
    #         ),
    #         "status": "gratitude"
    #     }

    # # === Handle goodbye ===
    # if any(bye in user_input for bye in BYE):
    #     return {
    #         "bot_message": (
    #             "👋 Take care of yourself! Remember, if symptoms persist, "
    #             "please visit a healthcare professional."
    #         ),
    #         "status": "goodbye"
    #     }

    # # Predict from symptom description 
    # X_input = preprocess_text(user_input)
    # if X_input is None:
    #     return {
    #         "bot_message": (
    #             "🤔 I couldn’t detect your condition it might be out of my scope of expertise.\n"
    #         "For more information please visit the link below"
                
    #         ),
    #         "status": "fallback"
    #     }

    # probs = model.predict(X_input)[0]
    # top_indices = np.argsort(probs)[::-1][:2]
    # predictions = [
    #     {
    #         "disease": label_encoder.inverse_transform([i])[0],
    #         "probability": float(probs[i])
    #     }
    #     for i in top_indices
    # ]

    # main_pred = predictions[0]
#     alt_pred = predictions[1]
#     confidence = main_pred["probability"]
#     CONF_THRESHOLD = 0.45

#     # === Low confidence fallback ===
#     if confidence < CONF_THRESHOLD:
#         return {
#             "bot_message": (
#                 "I’m not entirely sure what condition this might be. "
#                 "It could be something mild, but it’s best to consult a doctor for a precise diagnosis."
#             ),
#             "status": "uncertain",
#             "confidence": confidence
#         }

#     # Conversational response 
#     main_name = main_pred["disease"]

# bot_message = (
#         f"From what you’ve shared, you’re **most likely** experiencing **{main_name}**. "
#         f"It might also be **{alt_name}**, depending on how your symptoms progress.\n\n"
#         "Please monitor your symptoms and visit a healthcare provider for medical assistance."
#     )

#     return {
#         "bot_message": bot_message,
#         "main_prediction": main_name,
#         "alt_prediction": alt_name,
#         "status": "ok",
#         "confidence": confidence
#     }

# # === Health check route ===
# @app.get("/")
# def home():
#     return {"message": "Doc-AI backend is running smoothly!"}

In [33]:
# import random

# FOLLOW_UP_QUESTIONS = {
#     "pain": [
#         "Where exactly do you feel the pain?",
#         "How long have you had the pain?",
#         "Is it constant or does it come and go?"
#     ],
#     "cough": [
#         "Is it a dry cough or are you producing mucus?",
#         "Have you noticed any fever or shortness of breath?",
#         "How long have you been coughing?"
#     ],
#     "fever": [
#         "Do you know how high your temperature has been?",
#         "Have you experienced chills or sweating?",
#         "When did the fever start?"
#     ],
#     "fatigue": [
#         "Have you been sleeping normally?",
#         "Are you also experiencing dizziness or weakness?",
#         "Has the fatigue been getting worse over time?"
#     ]
# }

# def get_follow_up(user_input: str):
#     """Return a relevant clarifying question if a known symptom is found."""
#     for keyword, questions in FOLLOW_UP_QUESTIONS.items():
#         if keyword in user_input:
#             return random.choice(questions)
#     return random.choice([
#         "Can you describe any other symptoms you’re feeling?",
#         "When did these symptoms start?",
#         "Have you noticed any changes in your appetite or sleep?",
#     ])
# from lead_questions import get_follow_up